In [1]:
import sys
sys.path.append("..")

In [2]:

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from satreeverify.rf_utils import *

In [3]:
data = load_breast_cancer()
feature_names = data["feature_names"]

In [4]:
X = data['data']
y = data["target"] == 1
feature_names = data["feature_names"]
ntrees = 50
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)

In [5]:
clf.score(X, y)

0.9753954305799648

In [6]:
all_thresh = get_ens_thresh(clf)
data_ = disc_ens_data(X, all_thresh)
print(data_.shape)
data_.head()

(569, 289)


,12_1.5399999618530273,26_0.11295000091195107,26_0.34974999725818634,19_0.0016770000220276415,7_0.04988499917089939,13_42.19000053405762,27_0.1489500030875206,21_22.390000343322754,27_0.1419999971985817,7_0.049230000004172325,...,7_0.0506649985909462,22_106.54999923706055,13_18.09500026702881,22_106.0999984741211,22_106.04999923706055,23_889.8500061035156,27_0.14695000648498535,16_0.11137999966740608,13_79.82500076293945,1_22.454999923706055
0,True,True,True,True,True,True,True,False,True,True,...,True,True,True,True,True,True,True,False,True,False
1,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,False,False
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,False
3,True,True,True,True,True,False,True,True,True,True,...,True,False,True,False,False,False,True,False,False,False
4,True,True,True,True,True,True,True,False,True,True,...,True,True,True,True,True,True,True,False,True,False


In [7]:
var_x = create_var_x(all_thresh)

# Soft Solution

In [8]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.84394626, 0.15605374]])

In [9]:
s, c_weights = soft_attack(clf, sample, epsilon, var_x)

False
unknown


In [10]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.8725999420700458

In [11]:
clf.predict_proba(sample)[0][1]

0.15605373578615894

In [12]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
12,2.410000,1.539857,0.361055,"[1.3974999785423279, 1.5399999618530273]"
5,0.176500,0.112833,0.360718,"[0.09609000012278557, 0.11284999921917915]"
27,0.198600,0.127234,0.359347,"[0.11099999770522118, 0.12725000083446503]"
29,0.140500,0.092917,0.338668,"[0.09006500244140625, 0.09292000159621239]"
3,530.200000,370.029609,0.302094,"[-inf, 370.40000915527344]"
9,0.076920,0.055138,0.283174,"[0.05336499959230423, 0.05513999983668327]"
24,0.186200,0.136669,0.266010,"[0.10564999654889107, 0.13670000433921814]"
7,0.096010,0.071874,0.251388,"[0.05615000054240227, 0.07189000025391579]"
20,16.310000,12.587400,0.228240,"[-inf, 12.599999904632568]"
8,0.192500,0.149982,0.220871,"[0.1324000023305416, 0.14999999850988388]"


In [13]:
clf.predict_proba(x_adv_sample)[0][1]

0.8725999420700457

# Hard Solution

In [14]:
epsilon = 0.3

nbits = 8

index = 105
sample = X[index:index+1, :]

In [15]:
s, c_weights, seq_num = hard_attack(clf, sample, epsilon, var_x, nbits)

False
sat


In [16]:
clf.predict_proba(sample)[0][1]

0.15605373578615894

In [17]:
get_value(s.model(), seq_num, nbits , ntrees)

0.5127843137254903

In [18]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.51404736578939

In [19]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
15,0.029120,0.020646,0.291001,"[0.016705000307410955, 0.02065000031143427]"
8,0.192500,0.152180,0.209454,"[0.1324000023305416, 0.15219999849796295]"
27,0.198600,0.158992,0.199435,"[0.15129999816417694, 0.159000001847744]"
23,827.200000,667.731594,0.192781,"[-inf, 668.3999938964844]"
12,2.410000,2.851525,0.183206,"[2.85099995136261, 3.3765000104904175]"
13,34.660000,28.638780,0.173722,"[22.425000190734863, 28.645000457763672]"
9,0.076920,0.064146,0.166069,"[0.05513999983668327, 0.06415500119328499]"
21,22.400000,25.891494,0.155870,"[25.889999389648438, 27.385000228881836]"
0,13.110000,14.700345,0.121308,"[14.699999809265137, 15.045000076293945]"
24,0.186200,0.164541,0.116321,"[0.10564999654889107, 0.16459999978542328]"


In [20]:
clf.predict_proba(x_adv_sample)[0][1]

0.51404736578939